# Checking expressibility of ansatze

$\newcommand{\ket}[1]{\lvert #1 \rangle}$
$\newcommand{\bra}[1]{\langle #1 \rvert}$
$\newcommand{\braket}[1]{\langle #1 \rangle}$

[1905.10876](https://arxiv.org/abs/1905.10876) is much better on quantifying the capabilities of ansatze, but I didn't understand most of it. Instead I'll use the following much simpler quantity that appears in the paper: 
$$A = \int_{Haar} \ket{\psi} \bra{\psi} d\psi - \int_\theta U(\theta) \ket{s} \bra{s} U^\dagger(\theta) d\theta$$
where $\ket{s}$ means the starting state of our ansatz and $U$ is the ansatz. 

The Haar integral is a way to take the integral over a group like the unitaries. Instead, we'll approximate it with Monte-Carlo Numerical Integration (guide on [Jarrod McClean's blog](https://jarrodmcclean.com/integrating-over-the-unitary-group/)). 

Finally, we'll define the expressibility of an ansatz as the Frobenius norm of $A$. The lower it is, the more expressible the ansatz is. 

In [1]:
import numpy as np
import scipy as sp

import cirq
import openfermioncirq

from tools import utils

### Define Hubbard and get ansatz

Normal stuff for Hubbard...

In [2]:
from openfermion.utils import HubbardSquareLattice
# HubbardSquareLattice parameters
x_n = 2
y_n = 2
n_dofs = 1 # 1 degree of freedom for spin, this might be wrong. Having only one dof means ordered=False. 
periodic = 0 # Not sure what this is, periodic boundary conditions?
spinless = 0 # Has spin

lattice = HubbardSquareLattice(x_n, y_n, n_dofs=n_dofs, periodic=periodic, spinless=spinless)

from openfermion.hamiltonians import FermiHubbardModel
from openfermion.utils import SpinPairs
tunneling = [('neighbor', (0, 0), 1.)] # Not sure if this is right
interaction = [('onsite', (0, 0), 2., SpinPairs.DIFF)] # Not sure if this is right
potential = [(0, 1.)]
mag_field = 0. 
particle_hole_sym = False # Not sure if this is right

hubbard = FermiHubbardModel(lattice , tunneling_parameters=tunneling, interaction_parameters=interaction, 
                            potential_parameters=potential, magnetic_field=mag_field, 
                            particle_hole_symmetry=particle_hole_sym)

In [3]:
from openfermion import get_sparse_operator, get_ground_state
hub_sparse = get_sparse_operator(hubbard.hamiltonian())
print(hub_sparse.shape)
genergy, gstate = get_ground_state(hub_sparse)
print("Ground state energy: ", genergy)

(256, 256)
Ground state energy:  -6.828427124746191


In [4]:
from openfermioncirq import SwapNetworkTrotterHubbardAnsatz

ansatz = SwapNetworkTrotterHubbardAnsatz(x_n, y_n, 1., 2., periodic=False, iterations=5)

In [5]:
starting_state = gstate # CHANGE THIS
M = 500 
N = len(starting_state)

### Calculate the first integral

In [6]:
def random_unitary():
    """Generate uniformly random unitary from U(N). 
       Credit to Jarrod McClean"""
    Z = np.random.randn(N, N) + 1.0j * np.random.randn(N, N)
    [Q, R] = sp.linalg.qr(Z)
    D = np.diag(np.diagonal(R) / np.abs(np.diagonal(R)))
    return np.dot(Q, D)

In [7]:
random_unitary_density = 0

for i in range(M):
    psi = random_unitary() @ starting_state 
    random_unitary_density += utils.adjoint([psi]) @ [psi] # Sorry for weird formatting
random_unitary_density /= M

In [8]:
# Testing to make sure this works as expected
total_2 = 0

for i in range(M):
    psi = random_unitary() @ starting_state 
    total_2 += utils.adjoint([psi]) @ [psi]
total_2 /= M

# Should be low
np.linalg.norm(random_unitary_density - total_2)

0.06326688927544383

### Uniformly choose parameters

In [9]:
ansatz_density = 0
for i in range(M): 
    params = np.random.uniform(-2, 2, 15)
    circuit = cirq.resolve_parameters(ansatz.circuit, ansatz.param_resolver(params))
    final_state = circuit.final_wavefunction(starting_state)
    density = cirq.density_matrix_from_state_vector(final_state)
    ansatz_density += density 
ansatz_density /= M

In [10]:
np.linalg.norm(random_unitary_density - ansatz_density)

0.29492807521496

### Identity ansatz

It's hard to contextualize this result, so I'll try an ansatz which is just the identity $I$. This is a terrible ansatz and the score should reflect that. 

In [11]:
i_density = 0

for i in range(M):
    i_density += utils.adjoint([starting_state]) @ [starting_state]
i_density /= M

np.linalg.norm(random_unitary_density - i_density)

0.9990918483685354